# Latent Dirichlet Allocation

Following Mirko Draca and Carlo Schwarz selection of WVS questions and waves

In [85]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
import pandas as pd

df_wvs = pd.read_csv('../data/raw/wvs_ts_w1_w7.csv')

In [60]:
def recode_survey_responses(df_wvs, question_columns, neutral_values={3, 5}):
    """
    Following Draca & Schwarz (2024) methodology, this function recodes responses from WVS waves 4-7 
    into support and oppose indicators, imputes missing values, and filters countries based on response completeness.

    Parameters:
    - df_wvs (pd.DataFrame): Survey DataFrame.
    - question_columns (list): List of columns to transform.
    - neutral_values (set): Values representing neutrality (not used here but can be extended).
    
    Returns:
    - pd.DataFrame: Transformed DataFrame with support/oppose indicators and country/year.
    """
    # Step 1: Rename and filter for years >= 1999
    new_df = df_wvs.rename(columns={"COUNTRY_ALPHA": "country", "S020": "year", "S002VS": "wave"}) \
                   .loc[lambda df: df["year"] >= 1999] \
                   .copy()

    # Step 2: Filter for relevant waves
    valid_waves = [4, 5, 6, 7]
    df_filtered = new_df[new_df["wave"].isin(valid_waves)].copy()

    # Step 3: Find countries that appear in at least 3 of the 4 waves
    wave_counts = df_filtered.groupby("country")["wave"].nunique()
    eligible_countries = wave_counts[wave_counts >= 3].index
    df_final = df_filtered[df_filtered["country"].isin(eligible_countries)].copy()

    # Step 4: Impute missing values for each question column
    for col in question_columns:
        valid_values = df_final[df_final[col] >= 0][col]
        mean_value = valid_values.mean()
        df_final[col] = df_final[col].apply(lambda x: mean_value if x < 0 else x)

    # Step 5: Recode into support and oppose
    for col in question_columns:
        if col == "C002":  # 1–3 scale (agree-disagree)
            df_final[f"{col}_support"] = df_final[col].apply(lambda x: 1 if x == 1 else 0)
            df_final[f"{col}_oppose"] = df_final[col].apply(lambda x: 1 if x == 2 else 0)
        elif col in ["G006", "E069_01", "E069_02", "E069_04", "E069_05", "E069_06",
             "E069_07", "E069_08", "E069_13", "E069_17"]:  # 1–4 scale
             df_final[f"{col}_support"] = df_final[col].apply(lambda x: 1 if x in [1, 2] else 0)
             df_final[f"{col}_oppose"] = df_final[col].apply(lambda x: 1 if x in [3, 4] else 0)
        elif col in ["E036", "E037", "E039"] or "F1" in col:  # 1–10 scale
            df_final[f"{col}_support"] = df_final[col].apply(lambda x: 1 if x >= 6 else 0)
            df_final[f"{col}_oppose"] = df_final[col].apply(lambda x: 1 if x <= 4 else 0)
        else:  # Binary 0–1
            df_final[f"{col}_support"] = df_final[col]
            df_final[f"{col}_oppose"] = 1 - df_final[col]

    # Step 6: Keep only relevant columns
    interest_columns = ["country", "year"] + \
                       [f"{col}_support" for col in question_columns] + \
                       [f"{col}_oppose" for col in question_columns]

    return df_final[interest_columns]



In [69]:
# Test the function
question_columns = [
    "A124_02", "A124_06", "A124_07", "A124_08", "A124_09", "C002", "E036", "E037", "E039",
    "F114A", "F115", "F116", "F117", "F118", "F119", "F120", "F121", "F122", "F123", "G006", 
    "E069_01", "E069_02", "E069_04", "E069_05", "E069_06", "E069_07", "E069_08", "E069_13", "E069_17"
]

In [70]:
df_encoded = recode_survey_responses(df_wvs, question_columns)

/var/folders/vk/c6csf7ws6pj9wy406twfjkfw0000gn/T/ipykernel_31029/4208553163.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f"{col}_support"] = df_final[col].apply(lambda x: 1 if x in [1, 2] else 0)
/var/folders/vk/c6csf7ws6pj9wy406twfjkfw0000gn/T/ipykernel_31029/4208553163.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final[f"{col}_oppose"] = df_final[col].apply(lambda x: 1 if x in [3, 4] else 0)


In [76]:
## Exploring transformed file
# Check the first few rows of the transformed DataFrame
print(df_encoded.head())

     country  year  A124_02_support  A124_06_support  A124_07_support  \
7092     ARG  1999              0.0              0.0              0.0   
7093     ARG  1999              0.0              0.0              0.0   
7094     ARG  1999              0.0              0.0              1.0   
7095     ARG  1999              0.0              0.0              0.0   
7096     ARG  1999              0.0              0.0              0.0   

      A124_08_support  A124_09_support  C002_support  E036_support  \
7092              0.0              0.0             0             0   
7093              1.0              0.0             1             1   
7094              1.0              1.0             1             1   
7095              0.0              0.0             0             0   
7096              0.0              0.0             1             0   

      E037_support  ...  G006_oppose  E069_01_oppose  E069_02_oppose  \
7092             1  ...            0               0               1

In [77]:
### Split the data frame in the four waves

# Wave 4 1999 - 2004
df_wave4 = df_encoded[df_encoded['year'].between(1999, 2004)].copy()

# Wave 5 2005 - 2009
df_wave5 = df_encoded[df_encoded['year'].between(2005, 2009)].copy()

# Wave 6 2010 - 2014
df_wave6 = df_encoded[df_encoded['year'].between(2010, 2014)].copy()

# Wave 7 2017 - 2022
df_wave7 = df_encoded[df_encoded['year'].between(2017, 2022)].copy()

In [78]:
## Function to print dimensions of all the frames (wave 4-7)
for i, df in enumerate([df_wave4, df_wave5, df_wave6, df_wave7], start=4):
    print(f"Wave {i} dimensions: {df.shape}")

Wave 4 dimensions: (46100, 60)
Wave 5 dimensions: (57982, 60)
Wave 6 dimensions: (60783, 60)
Wave 7 dimensions: (62706, 60)


In [86]:
# Column names of the transformed DataFrames
print("Column names of the transformed DataFrames:")
for i, df in enumerate([df_wave4, df_wave5, df_wave6, df_wave7], start=4):
    print(f"Wave {i} columns: {df.columns.tolist()}")

Column names of the transformed DataFrames:
Wave 4 columns: ['country', 'year', 'A124_02_support', 'A124_06_support', 'A124_07_support', 'A124_08_support', 'A124_09_support', 'C002_support', 'E036_support', 'E037_support', 'E039_support', 'F114A_support', 'F115_support', 'F116_support', 'F117_support', 'F118_support', 'F119_support', 'F120_support', 'F121_support', 'F122_support', 'F123_support', 'G006_support', 'E069_01_support', 'E069_02_support', 'E069_04_support', 'E069_05_support', 'E069_06_support', 'E069_07_support', 'E069_08_support', 'E069_13_support', 'E069_17_support', 'A124_02_oppose', 'A124_06_oppose', 'A124_07_oppose', 'A124_08_oppose', 'A124_09_oppose', 'C002_oppose', 'E036_oppose', 'E037_oppose', 'E039_oppose', 'F114A_oppose', 'F115_oppose', 'F116_oppose', 'F117_oppose', 'F118_oppose', 'F119_oppose', 'F120_oppose', 'F121_oppose', 'F122_oppose', 'F123_oppose', 'G006_oppose', 'E069_01_oppose', 'E069_02_oppose', 'E069_04_oppose', 'E069_05_oppose', 'E069_06_oppose', 'E069_0

# LDA Application

In [87]:
import json
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dictionary from JSON file
with open("variable_dict.json", "r", encoding="utf-8") as file:
    variable_dict = json.load(file)

# Assuming df_encoded is the DataFrame with the survey data after recoding.
# Let's inspect df_encoded for LDA
print(df_encoded.head())

     country  year  A124_02_support  A124_06_support  A124_07_support  \
7092     ARG  1999              0.0              0.0              0.0   
7093     ARG  1999              0.0              0.0              0.0   
7094     ARG  1999              0.0              0.0              1.0   
7095     ARG  1999              0.0              0.0              0.0   
7096     ARG  1999              0.0              0.0              0.0   

      A124_08_support  A124_09_support  C002_support  E036_support  \
7092              0.0              0.0             0             0   
7093              1.0              0.0             1             1   
7094              1.0              1.0             1             1   
7095              0.0              0.0             0             0   
7096              0.0              0.0             1             0   

      E037_support  ...  G006_oppose  E069_01_oppose  E069_02_oppose  \
7092             1  ...            0               0               1

In [ ]:
### Este sí jaló

# Prepare Data for LDA (Remove Country & Year for now)
lda_data = df_encoded.drop(columns=["country", "year"])

# Fit LDA with 10 Ideological Groups
num_topics = 10
lda_model = LatentDirichletAllocation(n_components=num_topics,
                                    doc_topic_prior = 0.25 ,
                                    topic_word_prior = 0.1 ,
                                    learning_method='online', # online updating not batch faster
                                    learning_decay=0.7, # how soon parameters are forgotten
                                    learning_offset=10.0, #downweights early learning steppts
                                    max_iter=50, # max number of iterations default 10 (iterations in M step)
                                    batch_size=1000, #size of batch to use
                                    evaluate_every=-1, # evaluate perplexity -1 is off
                                    mean_change_tol=0.001, # stopping tolerance for updating in E-step
                                    max_doc_update_iter=300, # maximum number of iterations in E-step (iterations over batch)
                                    n_jobs=-1, #number of cpu to use
                                    random_state=25) #random state, original was in 42 
lda_matrix = lda_model.fit_transform(lda_data)

# Extract Topic-Feature Importance
feature_names = lda_data.columns
topic_words = pd.DataFrame(lda_model.components_, columns=feature_names)

# Normalize the Importance Scores
topic_words = topic_words.div(topic_words.sum(axis=1), axis=0)
topic_words = topic_words.T  # Transpose for better visualization
topic_words.columns = [f"Ideology_{i+1}" for i in range(num_topics)]

#### Este sí jaló

In [ ]:
# Display Top Issues for Each Ideology
top_issues = topic_words.apply(lambda x: x.nlargest(5).index.tolist(), axis=0)
top_issues

# Save the descriptive DataFrame to a CSV file
top_issues.to_csv('top_issues.csv', index=False)

,Ideology_1,Ideology_2,Ideology_3,Ideology_4,Ideology_5
0,A124_07_support,F119_oppose,F121_support,F115_support,A124_06_support
1,A124_09_support,F123_oppose,A124_02_oppose,F114A_support,A124_02_support
2,A124_08_support,F120_oppose,F117_oppose,F116_support,F119_oppose
3,F118_oppose,A124_02_oppose,A124_06_oppose,F121_support,F123_oppose
4,F119_oppose,A124_06_oppose,A124_09_oppose,F120_support,F117_oppose


In [29]:
### Folders for the results

# Subfolders exist
os.makedirs("../reports/lda_evaluation", exist_ok=True)
os.makedirs("../reports/top_issues", exist_ok=True)

# Example for saving evaluation results
#eval_df.to_csv(f"reports/lda_evaluation/wave{wave_number}_evaluation.csv", index=False)

# Later when saving top issues:
#top_issues_df.to_csv(f"reports/top_issues/wave{wave_number}_top_issues.csv", index=False)

In [88]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import os
from itertools import combinations

# Setup
num_folds = 10
topic_range = range(1, 11)  # K = 1 to 10
data_folder = "data"
results_folder = "../reports/model_evaluation"
os.makedirs(results_folder, exist_ok=True)

def compute_npmi(issue_list, test_data):
    """Compute NPMI for a list of top issues using test data."""
    N = len(test_data)
    epsilon = 1e-10

    binary_data = test_data[issue_list].astype(int)
    issue_counts = binary_data.sum(axis=0).to_dict()

    npmi_scores = []
    for i, j in combinations(issue_list, 2):
        p_i = issue_counts[i] / N
        p_j = issue_counts[j] / N
        p_ij = ((binary_data[i] & binary_data[j]).sum()) / N

        if p_ij > 0:
            pmi = np.log(p_ij / (p_i * p_j + epsilon))
            npmi = pmi / (-np.log(p_ij + epsilon))
            npmi_scores.append(npmi)

    return np.mean(npmi_scores) if npmi_scores else 0

def evaluate_model_with_npmi(model, test_data, train_data):
    """Evaluate LDA model using average NPMI across all topics."""
    feature_names = train_data.columns
    topic_words = pd.DataFrame(model.components_, columns=feature_names)
    top_issues_per_topic = topic_words.apply(lambda x: x.nlargest(5).index.tolist(), axis=1)

    topic_npmis = [compute_npmi(issue_list, test_data) for issue_list in top_issues_per_topic]
    return np.mean(topic_npmis)

In [89]:
def process_wave_from_df(wave_number, df_wave):
    print(f"\n🌊 Processing Wave {wave_number}")

    # Drop metadata columns so that LDA only uses issue variables
    lda_data = df_wave.drop(columns=["country", "year"])

    # Initialize K-Fold cross-validator
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # To store NPMI evaluation scores across folds and topic numbers
    evaluation_matrix = np.zeros((num_folds, len(topic_range)))
    npmi_scores_per_topic = {k: [] for k in topic_range}

    # Cross-validation loop
    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(lda_data)):
        print(f"📂 Fold {fold_idx + 1}/{num_folds}")

        train_data = lda_data.iloc[train_idx]
        test_data = lda_data.iloc[test_idx]

        for k_idx, n_topics in enumerate(topic_range):
            lda = LatentDirichletAllocation(
                n_components=n_topics,
                doc_topic_prior=0.25,
                topic_word_prior=0.1,
                learning_method='online',
                learning_decay=0.7,
                learning_offset=10.0,
                max_iter=20,
                batch_size=1000,
                evaluate_every=-1,
                mean_change_tol=0.001,
                max_doc_update_iter=100,
                n_jobs=-1,  # Uses all cores for this model
                random_state=25
            )
            lda.fit(train_data)

            # Evaluate using NPMI with held-out test data
            npmi_score = evaluate_model_with_npmi(lda, test_data, train_data)
            evaluation_matrix[fold_idx, k_idx] = npmi_score
            npmi_scores_per_topic[n_topics].append(npmi_score)

    # Compute the average NPMI across all folds for each topic count
    avg_npmis = {k: np.mean(npmi_scores_per_topic[k]) for k in topic_range}

    # Identify the optimal number of topics
    optimal_k = max(avg_npmis, key=avg_npmis.get)
    print(f"🏆 Optimal number of topics for Wave {wave_number}: K={optimal_k}")

    # Save evaluation results
    eval_df = pd.DataFrame(evaluation_matrix, columns=[f"K={k}" for k in topic_range])
    eval_df.to_csv(f"{results_folder}/wave{wave_number}_evaluation.csv", index=False)
    print(f"✅ Saved evaluation results for Wave {wave_number}")


# Run for waves 4–7 sequentially to avoid CPU overload
for wave_number, df in zip(range(4, 8), [df_wave4, df_wave5, df_wave6, df_wave7]):
    process_wave_from_df(wave_number, df)

print("\n🎯 Done! All waves processed using NPMI evaluation.")


🌊 Processing Wave 4
📂 Fold 1/10
📂 Fold 2/10
📂 Fold 3/10
📂 Fold 4/10
📂 Fold 5/10
📂 Fold 6/10
📂 Fold 7/10
📂 Fold 8/10
📂 Fold 9/10
📂 Fold 10/10
🏆 Optimal number of topics for Wave 4: K=4
✅ Saved evaluation results for Wave 4

🌊 Processing Wave 5
📂 Fold 1/10
📂 Fold 2/10
📂 Fold 3/10
📂 Fold 4/10
📂 Fold 5/10
📂 Fold 6/10
📂 Fold 7/10
📂 Fold 8/10
📂 Fold 9/10
📂 Fold 10/10
🏆 Optimal number of topics for Wave 5: K=7
✅ Saved evaluation results for Wave 5

🌊 Processing Wave 6
📂 Fold 1/10
📂 Fold 2/10
📂 Fold 3/10
📂 Fold 4/10
📂 Fold 5/10
📂 Fold 6/10
📂 Fold 7/10
📂 Fold 8/10
📂 Fold 9/10
📂 Fold 10/10
🏆 Optimal number of topics for Wave 6: K=3
✅ Saved evaluation results for Wave 6

🌊 Processing Wave 7
📂 Fold 1/10
📂 Fold 2/10
📂 Fold 3/10
📂 Fold 4/10
📂 Fold 5/10
📂 Fold 6/10
📂 Fold 7/10
📂 Fold 8/10
📂 Fold 9/10
📂 Fold 10/10
🏆 Optimal number of topics for Wave 7: K=3
✅ Saved evaluation results for Wave 7

🎯 Done! All waves processed using NPMI evaluation.


In [93]:
# Extract Topic-Feature Importance
feature_names = lda_data.columns
topic_words = pd.DataFrame(lda_model.components_, columns=feature_names)

# Normalize the Importance Scores
topic_words = topic_words.div(topic_words.sum(axis=1), axis=0)
topic_words = topic_words.T  # Transpose for better visualization
topic_words.columns = [f"Ideology_{i+1}" for i in range(num_topics)]

In [94]:
# Display Top Issues for Each Ideology
top_issues = topic_words.apply(lambda x: x.nlargest(10).index.tolist(), axis=0)
top_issues

# Save the descriptive DataFrame to a CSV file
top_issues.to_csv('top_issues.csv', index=False)

In [95]:
top_issues

,Ideology_1,Ideology_2,Ideology_3,Ideology_4,Ideology_5,Ideology_6,Ideology_7,Ideology_8,Ideology_9,Ideology_10
0,F114A_support,G006_oppose,E069_06_oppose,A124_09_support,F118_support,E069_17_support,A124_07_support,E069_02_oppose,A124_07_oppose,F122_support
1,F115_support,E069_01_oppose,E069_17_oppose,F118_oppose,E039_support,E069_08_support,A124_09_support,E069_06_oppose,F120_oppose,F121_support
2,F116_support,E069_02_oppose,E069_04_support,F120_oppose,F121_support,E069_06_support,F118_oppose,E069_07_oppose,A124_09_oppose,A124_09_oppose
3,F117_support,E037_support,E069_13_support,F119_oppose,A124_08_oppose,E069_07_support,E039_oppose,E069_08_oppose,F118_oppose,F118_support
4,F122_support,E069_07_oppose,E069_05_support,A124_08_support,A124_09_oppose,E069_13_support,F119_oppose,E069_04_oppose,F119_oppose,F114A_oppose
5,F123_support,E069_13_oppose,E069_07_oppose,F123_oppose,A124_07_oppose,E069_05_support,F120_oppose,E069_17_oppose,F122_oppose,F120_support
6,F121_support,A124_02_oppose,E069_08_oppose,F117_oppose,E036_support,E069_04_support,A124_06_support,E069_05_oppose,F123_oppose,F117_oppose
7,A124_08_support,E069_08_oppose,E069_02_support,F122_oppose,F119_support,E069_02_support,F117_oppose,E069_13_oppose,F117_oppose,A124_07_oppose
8,A124_07_support,A124_06_oppose,E069_01_support,F116_oppose,F120_support,A124_02_oppose,F115_oppose,E069_01_oppose,F116_oppose,F116_oppose
9,A124_09_support,F117_oppose,A124_02_oppose,A124_02_oppose,E069_02_oppose,A124_06_oppose,F123_oppose,A124_02_oppose,E039_oppose,E039_oppose


In [ ]:
### Mergin back the results to country and year 
for wave_df in [df_wave4, df_wave5, df_wave6, df_wave7]:
    lda_data = wave_df.drop(columns=["country", "year"])
    lda.fit(lda_data)
    topic_dist = lda.transform(lda_data)
    topic_df = pd.DataFrame(topic_dist, columns=[f"Topic_{i+1}" for i in range(topic_dist.shape[1])])
    merged = pd.concat([wave_df.reset_index(drop=True), topic_df], axis=1)

    # Optionally save it
    year_range = f"{wave_df['year'].min()}_{wave_df['year'].max()}"
    merged.to_csv(f"../reports/merged_wave_{year_range}.csv", index=False)